In [4]:
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
from pyblock2 import tools
from block2 import SU2 as block2_SU2
from block2 import SZ as block2_SZ
from pathlib import Path
import CAS.dmrghandler.src.dmrghandler.dmrg_calc_prepare
import CAS.dmrghandler.src.dmrghandler.qchem_dmrg_calc
import CAS.dmrghandler.src.dmrghandler as dmrghandler
from DMRG_simulation.dmrg_simulation import get_ground_state_fci
import numpy as np
import CAS_Cropping.saveload_utils as sl
import CAS_Cropping.ferm_utils as feru
import CAS_Cropping.var_utils as varu
import openfermion as of

In [5]:

mps_dir = Path("test_mps/dmrg_loop_001_ket_optimized")
fcidump = "fcidumps_test/fcidump.36_1ru_II_2pl_{'default'_'6-31_G(dp)''Ru'_'lanl2tz'}_tbt_1_block_test"
symmetry_type = SymmetryTypes.SU2
stack_mem = 10*1024*1024*1024
stack_mem_ratio = 0.5
num_threads = 3
temp_dir = Path(mps_dir)
restart_dir=Path("./temp")

In [6]:
# def loadMPSfromDir(mps_info: MPSInfo,  mpsSaveDir:str, MPI:MPICommunicator=None) -> MPS:
#         """  Load MPS from directory
#         :param mps_info: If None, MPSInfo will be read from mpsSaveDir/mps_info.bin
#         :param mpsSaveDir: Directory where the MPS has been saved
#         :param MPI: MPI class (or None)
#         :return: MPS state
#         """
driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SU2, n_threads=4)
driver.read_fcidump(filename=fcidump, pg='d2h')
driver.initialize_system(n_sites=driver.n_sites, n_elec=driver.n_elec,
                         spin=driver.spin, orb_sym=driver.orb_sym)
tools.init(block2_SU2)
mps = tools.loadMPSfromDir(mps_info=None, mpsSaveDir=mps_dir)
csfs, coeffs = driver.get_csf_coefficients(mps, cutoff=1e-10, iprint=1)

symmetrize error =  0.0
mps center changed (temporarily)
dtrie finished 0.014281165999932455
Number of CSF =          6 (cutoff =     1e-10)
Sum of weights of included CSF =    1.000000000000000

CSF          0 2202222  =   -0.999632357118269
CSF          1 2022222  =    0.022422340198507
CSF          2 0222222  =    0.012649796455178
CSF          3 2+-2222  =    0.008496266234389
CSF          4 +-22222  =   -0.000423661442038
CSF          5 +2-2222  =   -0.000076700030599


In [166]:
print(coeffs)

[-6.54420995e-02  5.98087691e-03  6.22097954e-02 -5.98087768e-03
 -9.55066378e-03 -5.98085769e-03 -7.17602636e-02  9.90471942e-01
 -3.97898091e-03  8.34564188e-09 -3.97889641e-03 -7.17606048e-02
  5.98086868e-03 -9.55091271e-03 -8.84026339e-03 -1.15727969e-08
 -8.84038889e-03 -1.93417094e-07 -1.68314578e-08 -3.68488849e-07]


In [144]:
cmpct_csfs = []
cmpct_coeffs = []

In [151]:
cmpct_csfs.append(csfs[19])
cmpct_coeffs.append(coeffs[19])
print(cmpct_csfs)

[array([3, 3, 3, 3, 3, 3, 0], dtype=uint8), array([3, 3, 3, 3, 3, 0, 3], dtype=uint8), array([3, 3, 0, 3, 3, 3, 3], dtype=uint8), array([3, 3, 3, 0, 3, 3, 3], dtype=uint8), array([3, 3, 3, 3, 0, 3, 3], dtype=uint8), array([0, 3, 3, 3, 3, 3, 3], dtype=uint8), array([3, 0, 3, 3, 3, 3, 3], dtype=uint8)]


In [120]:
cmpct_csfs.pop()
cmpct_coeffs.pop()

0.0002510419984715899

In [152]:
normalize_constant = 0
for coeff in cmpct_coeffs:
    normalize_constant += coeff**2
print(normalize_constant)

0.999999933078537


In [153]:
corrected_coeffs = []
for coeff in cmpct_coeffs:
    corrected_coeffs.append(coeff * normalize_constant)

In [142]:
print(corrected_coeffs)

[-0.9999945240402133, 0.0032387201068204852, 0.00036426867097076284, 0.0003685321186918828, 0.00034842132160782627, 0.0002767238894035529, 0.00025104201527171]


In [160]:
cmpct_mps = driver.get_mps_from_csf_coefficients(cmpct_csfs, cmpct_coeffs, tag="cmpct_csf", dot=1)
csfs, coeffs = driver.get_csf_coefficients(cmpct_mps, cutoff=1e-10, iprint=1)


mps center changed (temporarily)
dtrie finished 0.0011498329986352473
Number of CSF =          7 (cutoff =     1e-10)
Sum of weights of included CSF =    0.999999933078537

CSF          0 2222220  =   -0.999994457119061
CSF          1 2222202  =    0.003238719894037
CSF          2 2202222  =    0.000368532093908
CSF          3 2220222  =    0.000364268646469
CSF          4 2222022  =    0.000348421297868
CSF          5 0222222  =    0.000276723871789
CSF          6 2022222  =    0.000251041999130


In [127]:
print(coeffs)

[-7.60030895e-05 -2.15013732e-05  3.48421298e-04 -6.71666458e-06
 -5.36766959e-05  3.23871989e-03  2.51877834e-05 -9.99994457e-01
 -9.18485211e-06 -5.49749515e-05 -5.76404615e-05  3.64268647e-04
  3.25663338e-06  2.34806290e-05 -1.13531264e-04  3.57741564e-05
  3.68532094e-04  7.99830264e-06  5.24419465e-05  2.51041998e-04
  4.47396794e-05  1.26288860e-05 -1.58405115e-04 -2.35093428e-05
  5.61317159e-05  3.61818328e-05  2.76723871e-04 -2.96969325e-05]


In [103]:
print(csfs)

[[3 3 3 1 3 2 3]
 [3 3 3 1 3 3 2]
 [3 3 3 3 0 3 3]
 [3 3 3 3 1 2 3]
 [3 3 3 3 1 3 2]
 [3 3 3 3 3 0 3]
 [3 3 3 3 3 1 2]
 [3 3 3 3 3 3 0]
 [3 3 1 3 2 3 3]
 [3 3 1 3 3 2 3]
 [3 3 1 3 3 3 2]
 [3 3 3 0 3 3 3]
 [3 3 3 1 2 3 3]
 [3 1 3 3 2 3 3]
 [3 1 3 3 3 2 3]
 [3 1 3 3 3 3 2]
 [3 3 0 3 3 3 3]
 [3 3 1 2 3 3 3]
 [3 1 3 2 3 3 3]
 [3 0 3 3 3 3 3]
 [3 1 2 3 3 3 3]
 [1 3 3 3 2 3 3]
 [1 3 3 3 3 2 3]
 [1 3 3 3 3 3 2]
 [1 3 3 2 3 3 3]
 [1 3 2 3 3 3 3]
 [0 3 3 3 3 3 3]
 [1 2 3 3 3 3 3]]


In [161]:
(
    one_body_tensor,
    two_body_tensor,
    nuc_rep_energy,
    num_orbitals,
    num_spin_orbitals,
    num_electrons,
    two_S,
    two_Sz,
    orb_sym,
    extra_attributes,
    ) = dmrghandler.dmrg_calc_prepare.load_tensors_from_fcidump(data_file_path=fcidump, molpro_orbsym_convention=True)

Parsing fcidumps_test/fcidump.36_1ru_II_2pl_{'default'_'6-31_G(dp)''Ru'_'lanl2tz'}_tbt_1_test


In [162]:
bond_dims = [1] * 4 + [10] * 4
noises = [1e-4] * 4 + [1e-5] * 4 + [0]
thrds = [1e-10] * 8

mpo = driver.get_qc_mpo(h1e=one_body_tensor, g2e=two_body_tensor, ecore=nuc_rep_energy, iprint=0)
energy = driver.dmrg(mpo, mps, n_sweeps=20, bond_dims=bond_dims, noises=noises,
    thrds=thrds, iprint=1)

ener = driver.expectation(mps, mpo, mps)
print(ener)

bra = driver.get_random_mps(tag="GS5", bond_dim=250, nroots=1)
applied_mps = driver.multiply(bra ,mpo, cmpct_mps)
print(applied_mps)

variance = np.sqrt(abs(applied_mps ** 2 - ener ** 2))
print(f"Variance: {variance}")
print(f"Variance ratio: {variance / ener}")


Sweep =    0 | Direction =  forward | Bond dimension =    1 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      0.019 | E =   -3748.2039936058 | DW = 6.05e-05

Sweep =    1 | Direction = backward | Bond dimension =    1 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      0.033 | E =   -3748.2039862955 | DE = 7.31e-06 | DW = 8.34e-05

Sweep =    2 | Direction =  forward | Bond dimension =    1 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      0.047 | E =   -3748.2039862955 | DE = 0.00e+00 | DW = 6.05e-05

Sweep =    3 | Direction = backward | Bond dimension =    1 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      0.062 | E =   -3748.2039862955 | DE = 0.00e+00 | DW = 8.34e-05

Sweep =    4 | Direction =  forward | Bond dimension =   10 | Noise =  1.00e-05 | Dav threshold =  1.00e-10
Time elapsed =      0.077 | E =   -3748.2039862955 | DE = 0.00e+00 | DW = 0.00e+00

Sweep =    5 | Direction = backward | Bond dimension = 

In [2]:
mol = 'h2'
Hf = sl.load_fermionic_hamiltonian(mol, prefix="../CAS/")
print(Hf)

0.52917721092 [] +
-1.110844179883727 [0^ 0] +
0.31320124976475894 [0^ 0^ 0 0] +
0.09839529174273512 [0^ 0^ 2 2] +
0.31320124976475894 [0^ 1^ 1 0] +
0.09839529174273512 [0^ 1^ 3 2] +
0.09839529174273512 [0^ 2^ 0 2] +
0.3108533815598566 [0^ 2^ 2 0] +
0.09839529174273512 [0^ 3^ 1 2] +
0.3108533815598566 [0^ 3^ 3 0] +
0.31320124976475894 [1^ 0^ 0 1] +
0.09839529174273512 [1^ 0^ 2 3] +
-1.110844179883727 [1^ 1] +
0.31320124976475894 [1^ 1^ 1 1] +
0.09839529174273512 [1^ 1^ 3 3] +
0.09839529174273512 [1^ 2^ 0 3] +
0.3108533815598566 [1^ 2^ 2 1] +
0.09839529174273512 [1^ 3^ 1 3] +
0.3108533815598566 [1^ 3^ 3 1] +
0.31085338155985665 [2^ 0^ 0 2] +
0.09839529174273512 [2^ 0^ 2 0] +
0.31085338155985665 [2^ 1^ 1 2] +
0.09839529174273512 [2^ 1^ 3 0] +
-0.5891210037060828 [2^ 2] +
0.09839529174273512 [2^ 2^ 0 0] +
0.3265353734712869 [2^ 2^ 2 2] +
0.09839529174273512 [2^ 3^ 1 0] +
0.3265353734712869 [2^ 3^ 3 2] +
0.31085338155985665 [3^ 0^ 0 3] +
0.09839529174273512 [3^ 0^ 2 1] +
0.3108533815598566

In [35]:
from DMRG_simulation.format_dmrg.format_dmrg_param import get_dmrg_param, get_dmrg_process_param

mol = 'h2'
Hf = sl.load_fermionic_hamiltonian(mol, prefix="../CAS/")
spin_orb = of.count_qubits(Hf)
print("Spin orb:", spin_orb)
two_body = feru.get_chemist_tbt(Hf, spin_orb, spin_orb=True)
one_body = feru.get_obt(Hf, spin_orb, spin_orb=True)
print(two_body)
spatial_obt, spatial_tbt, spin_symm_broken = dmrghandler.dmrg_calc_prepare.spinorbitals_to_orbitals(one_body, two_body)
print(spin_symm_broken)

print(spatial_tbt.shape[0])

default_final_bond_dim = 1
default_sweep_schedule_bond_dims = [default_final_bond_dim] * 4 + [
    default_final_bond_dim
] * 4
default_sweep_schedule_noise = [1e-4] * 4 + [1e-5] * 4 + [0]
default_sweep_schedule_davidson_threshold = [1e-10] * 8

init_state_bond_dimension = 1
max_num_sweeps = 200
energy_convergence_threshold = 1e-8
sweep_schedule_bond_dims = default_sweep_schedule_bond_dims
sweep_schedule_noise = default_sweep_schedule_noise
sweep_schedule_davidson_threshold = (
    default_sweep_schedule_davidson_threshold
)
nuc_rep_energy = 0
    
num_orbitals = 2
num_electrons = 2
num_unpaired = 0
multiplicity = 1
dmrg_process_param = get_dmrg_process_param(
    init_state_bond_dimension, max_num_sweeps, energy_convergence_threshold,
    sweep_schedule_bond_dims, sweep_schedule_noise,
    sweep_schedule_davidson_threshold)
dmrg_param = get_dmrg_param(
    num_orbitals, num_electrons, num_unpaired,
    multiplicity, dmrg_process_param)
    
result = dmrghandler.qchem_dmrg_calc.single_qchem_dmrg_calc(spatial_obt, spatial_tbt, dmrg_param)
mps = result['ket_optimized']
print(result)

Spin orb: 4
[[[[0.31320125 0.         0.         0.        ]
   [0.         0.31320125 0.         0.        ]
   [0.         0.         0.31085338 0.        ]
   [0.         0.         0.         0.31085338]]

  [[0.         0.         0.         0.        ]
   [0.         0.         0.         0.        ]
   [0.         0.         0.         0.        ]
   [0.         0.         0.         0.        ]]

  [[0.         0.         0.09839529 0.        ]
   [0.         0.         0.         0.09839529]
   [0.09839529 0.         0.         0.        ]
   [0.         0.09839529 0.         0.        ]]

  [[0.         0.         0.         0.        ]
   [0.         0.         0.         0.        ]
   [0.         0.         0.         0.        ]
   [0.         0.         0.         0.        ]]]


 [[[0.         0.         0.         0.        ]
   [0.         0.         0.         0.        ]
   [0.         0.         0.         0.        ]
   [0.         0.         0.         0.        

In [49]:
from openfermion.linalg import expectation, get_number_preserving_sparse_operator

state1 = np.array([1, 0, 0, 1])
state2 = np.array([1, 1, 0, 1])
mol = 'h2'
Hf = sl.load_fermionic_hamiltonian(mol, prefix="../CAS/")
sparse = get_number_preserving_sparse_operator(
    Hf,
    4,
    2,
    spin_preserving=True,
    reference_determinant=None,
    excitation_level=None
)
result = np.dot(np.conjugate(state2), sparse * state1)
print(result)
print(sparse)
result2 =  np.dot(np.conjugate(state1), sparse * sparse * state1)
print(result2)

-0.6685215318965874
  (0, 0)	-1.0661086493179361
  (3, 0)	0.19679058348547024
  (1, 1)	-0.5490812095500965
  (2, 1)	-0.19679058348547024
  (1, 2)	-0.19679058348547024
  (2, 2)	-0.5490812095500965
  (0, 3)	0.19679058348547024
  (3, 3)	0.004005950450408302
0.7960331476233619


In [79]:
print(gs)

[-2.05916709e-16-2.06042220e-16j  3.90120943e-18+1.81811978e-17j
  1.25533354e-19+2.34518980e-18j  1.30037065e-01+1.17572708e-01j
 -1.03852866e-16+9.80231344e-17j  1.65068906e-16+2.12994444e-16j
  1.59503781e-17+1.22779328e-16j -8.13957203e-17+8.46570692e-18j
  8.76483431e-17-3.77119759e-18j  8.67597385e-17+2.82796205e-16j
 -4.16277114e-17-1.66028970e-16j -6.95148222e-17-3.20427425e-17j
 -7.30275181e-01-6.60276594e-01j  7.90996627e-17-2.71820843e-17j
 -1.25434559e-16+1.94747454e-16j  6.42926034e-17-1.71832276e-16j]


In [26]:
mol = 'h2'
Hf = sl.load_fermionic_hamiltonian(mol, prefix="../CAS/")
ground_energy, ground_state = get_ground_state_fci(mol, path="../CAS/")

<class 'openfermion.ops.operators.fermion_operator.FermionOperator'>
<class 'openfermion.ops.operators.fermion_operator.FermionOperator'>


In [27]:
print(ground_state)
print(ground_energy)

[-3.45875348e-18+5.43180044e-17j -5.06623854e-17+3.03262077e-17j
  7.44937116e-17-1.14496158e-16j -1.31260206e-01-1.16205586e-01j
  5.59967899e-18+1.81510941e-16j  1.96307051e-16-2.92076209e-16j
  9.18412486e-17+4.32681988e-16j -8.75560026e-17-1.05668652e-16j
  3.15028986e-16+1.08544494e-16j -2.30507331e-16+3.14855764e-16j
 -2.46627595e-16-5.02624687e-17j -1.53619265e-16-3.46519576e-17j
  7.37144219e-01+6.52598973e-01j -4.07008212e-16+4.54092407e-16j
 -6.35019038e-17-3.44911793e-16j  7.78062542e-17+4.75163862e-17j]
-1.6303275411526181


In [66]:
print(result.keys())

dict_keys(['dmrg_driver', 'dmrg_ground_state_energy', 'initial_ket', 'initial_bond_dims', 'ket_optimized', 'dmrg_discarded_weight', 'sweep_bond_dims', 'sweep_max_discarded_weight', 'sweep_energies', 'wall_make_driver_time_s', 'cpu_make_driver_time_s', 'wall_driver_initialize_system_time_s', 'cpu_driver_initialize_system_time_s', 'wall_reorder_integrals_time_s', 'cpu_reorder_integrals_time_s', 'wall_get_qchem_hami_mpo_time_s', 'cpu_get_qchem_hami_mpo_time_s', 'wall_generate_initial_mps_time_s', 'cpu_generate_initial_mps_time_s', 'wall_copy_mps_time_s', 'cpu_copy_mps_time_s', 'wall_dmrg_optimization_time_s', 'cpu_dmrg_optimization_time_s', 'wall_single_qchem_dmrg_calc_time_s', 'cpu_single_qchem_dmrg_calc_time_s'])


In [79]:
driver = DMRGDriver(
        stack_mem=stack_mem,
        scratch=str(temp_dir),
        clean_scratch=True,  # Default value
        restart_dir=str(restart_dir),
        n_threads=num_threads,
        # n_mkl_threads=n_mkl_threads,  # Default value is 1
        symm_type=symmetry_type,
        mpi=None,  # Default value
        stack_mem_ratio=stack_mem_ratio,  # Default value 0.4
        fp_codec_cutoff=1e-16,  # Default value 1e-16,
        
    )

In [80]:
driver.initialize_system(n_sites=num_orbitals, n_elec=num_electrons, spin=num_electrons, orb_sym=None)

In [81]:
mpo = driver.get_qc_mpo(h1e=one_body_tensor, g2e=two_body_tensor, ecore=nuc_rep_energy, iprint=0)

In [57]:

energy = driver.dmrg(mpo, mps, n_sweeps=20, bond_dims=bond_dims, noises=noises,
    thrds=thrds, iprint=1)


Sweep =    0 | Direction = backward | Bond dimension =    1 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      0.063 | E =   -3570.4283181384 | DW = 8.22e-05

Sweep =    1 | Direction =  forward | Bond dimension =    1 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      0.083 | E =   -3570.4283108291 | DE = 7.31e-06 | DW = 6.05e-05

Sweep =    2 | Direction = backward | Bond dimension =    1 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      0.097 | E =   -3570.4283108291 | DE = 0.00e+00 | DW = 8.16e-05

Sweep =    3 | Direction =  forward | Bond dimension =    1 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
Time elapsed =      0.112 | E =   -3570.4283108291 | DE = 0.00e+00 | DW = 6.05e-05

Sweep =    4 | Direction = backward | Bond dimension =    3 | Noise =  1.00e-05 | Dav threshold =  1.00e-10
Time elapsed =      0.136 | E =   -3570.4283176063 | DE = -6.78e-06 | DW = 2.01e-09

Sweep =    5 | Direction =  forward | Bond dimension =

In [58]:
mps.info.bond_dim

36

In [86]:
csfs, coeffs = driver.get_csf_coefficients(mps, cutoff=1e-10, iprint=1)

dtrie finished 0.0029057499996270053
Number of CSF =         54 (cutoff =     1e-10)
Sum of weights of included CSF =    1.000000000000000

CSF          0 2222200  =   -0.999980504521635
CSF          1 2222+-0  =   -0.003440485114342
CSF          2 +2222-0  =   -0.002926072873364
CSF          3 2222020  =    0.002133508140008
CSF          4 0222220  =    0.001830590034143
CSF          5 2220220  =    0.001820324356732
CSF          6 2022220  =    0.001806748030014
CSF          7 2202220  =    0.001660122416371
CSF          8 2222+0-  =    0.000802282635776
CSF          9 +22220-  =   -0.000499831097528
CSF         10 2222002  =    0.000343756742188
CSF         11 0222202  =    0.000294949737767
CSF         12 2220202  =    0.000293295703383
CSF         13 2022202  =    0.000291108248010
CSF         14 2202202  =    0.000267483522928
CSF         15 +222-20  =   -0.000093079613559
CSF         16 22220+-  =    0.000036459302765
CSF         17 02222+-  =    0.000031282766183
CSF         18

In [93]:
ener = driver.expectation(mps, mpo, mps)
print(ener)

-3865.717206234868


In [94]:
bra = driver.get_random_mps(tag="GS4", bond_dim=250, nroots=1)
applied_mps = driver.multiply(bra ,mpo, mps)

In [97]:
variance = np.sqrt(abs(applied_mps ** 2 - ener ** 2))
print(f"Variance: {variance}")
print(f"Variance ratio: {variance / ener}")

Variance: 0.01169066242027463
Variance ratio: -3.0241897677924307e-06


In [300]:
csfs, coeffs = driver.get_csf_coefficients(mps, cutoff=0.05, iprint=1)

dtrie finished 0.005632624999634572
Number of CSF =         12 (cutoff =      0.05)
Sum of weights of included CSF =    0.996244379611767

CSF          0 22+22+22222--0  =    0.614561088123480
CSF          1 22+22+2222-2-0  =   -0.560569826498747
CSF          2 22++2222222--0  =    0.330779569868118
CSF          3 22++222222-2-0  =   -0.301719470486626
CSF          4 22+22+222-22-0  =   -0.177664504444928
CSF          5 22+22+22222-0-  =    0.153776143303813
CSF          6 22+22+2222-20-  =   -0.140266391148645
CSF          7 22++22222-22-0  =   -0.095625625339491
CSF          8 22++2222222-0-  =    0.082768023425188
CSF          9 22++222222-20-  =   -0.075496573778813
CSF         10 22+2+222222--0  =   -0.063080199355577
CSF         11 22+2+22222-2-0  =    0.057538391368438


In [22]:
print(ener)

-5843.712701827251


In [ ]:
ordm1 = driver.get_orbital_entropies(mps, orb_type=1)
ordm2 = driver.get_orbital_entropies(mps, orb_type=2)
minfo = 0.5 * (ordm1[:, None] + ordm1[None, :] - ordm2) * (1 - np.identity(len(ordm1)))

In [8]:
mps_to_use = driver.load_mps(tag="ket_optimized", nroots=1)

AttributeError: 'NoneType' object has no attribute 'opf'